## Sustainability Hangman
### A traditional hangman game enhanced with sustainability terms and uses three types of AI guess strategies
#### CISC 440: Artificial Intelligience
#### Daniel Bauer, Connor Niznick, Porter Provan, Breanna Ranglall

In [2]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import string
import math
import random
from collections import Counter
import string
import math

Load in sustainability words list and filter out double words or words with hypens

In [4]:
def load_cleaned_words(filepath):
    with open(filepath, 'r') as f:
        raw_words = f.readlines()
    return [
        word.strip().lower()
        for word in raw_words
        if word.strip()  # Skip empty or whitespace-only lines
        and all(
            c.isalpha() or c in "- "  # Allow only letters, hyphens, and spaces
            for c in word.strip()
        )
    ]

word_list = load_cleaned_words("sustainability_words.txt")

Update game board

In [6]:
max_attempts = 6  # Max incorrect guesses allowed before game over

# Draw the hangman figure based on number of mistakes
def draw_hangman(canvas, mistakes):
    canvas.delete("all")
    canvas.create_line(20, 180, 100, 180)  # base
    canvas.create_line(60, 180, 60, 20)    # pole
    canvas.create_line(60, 20, 120, 20)    # top beam
    canvas.create_line(120, 20, 120, 40)   # rope
    if mistakes > 0:
        canvas.create_oval(110, 40, 130, 60)  # head
    if mistakes > 1:
        canvas.create_line(120, 60, 120, 100)  # body
    if mistakes > 2:
        canvas.create_line(120, 70, 100, 90)   # left arm
    if mistakes > 3:
        canvas.create_line(120, 70, 140, 90)   # right arm
    if mistakes > 4:
        canvas.create_line(120, 100, 100, 130) # left leg
    if mistakes > 5:
        canvas.create_line(120, 100, 140, 130) # right leg

# Update the displayed version of the secret word
def update_display(display_var, secret_word, guesses):
    # Show letters that are guessed, but always show hyphens and spaces
    display = [
        c if (c in guesses or c in "- ") else "_"
        for c in secret_word
    ]
    display_var.set(" ".join(display))

AI Strategies

In [8]:
import math
import string

# Greedy: pick the most common unguessed letter in possible words
def greedy_guess(possible_words, guessed):
    freq = {}  # Dictionary to track how many words contain each unguessed letter
    for word in possible_words:
        for letter in set(word):  # Use set to avoid double-counting letters in the same word
            if letter not in guessed:
                freq[letter] = freq.get(letter, 0) + 1  # Increment count for the letter
    # Return the letter that appears in the most words among unguessed letters
    return max(freq, key=freq.get) if freq else None


# Frequency of letters in English (used for UCS heuristic)
letter_freq = {
    'a': 8.2, 'b': 1.5, 'c': 2.8, 'd': 4.3, 'e': 12.7,
    'f': 2.2, 'g': 2.0, 'h': 6.1, 'i': 7.0, 'j': 0.15,
    'k': 0.77, 'l': 4.0, 'm': 2.4, 'n': 6.7, 'o': 7.5,
    'p': 1.9, 'q': 0.095, 'r': 6.0, 's': 6.3, 't': 9.1,
    'u': 2.8, 'v': 0.98, 'w': 2.4, 'x': 0.15, 'y': 2.0,
    'z': 0.074
}


# UCS: choose letter with the lowest "cost" = 1 / frequency
# Lower cost implies more common letters, which are more likely to appear
def ucs_guess(possible_words, guessed):
    # Get list of letters that haven't been guessed yet
    unguessed = [l for l in letter_freq if l not in guessed]

    # Compute cost for each unguessed letter using inverse of frequency
    costs = {l: 1 / letter_freq[l] for l in unguessed}

    # Find the minimum cost (most common letter)
    min_cost = min(costs.values())

    # Gather all letters with that minimum cost (handle ties)
    best_letters = [l for l in costs if abs(costs[l] - min_cost) < 1e-6]

    # If multiple letters have the same cost, choose the one that occurs most in the possible words
    if len(best_letters) > 1:
        word_blob = "".join(possible_words)
        freq = {l: word_blob.count(l) for l in best_letters}
        return max(freq, key=freq.get)

    return best_letters[0]  # Return the cheapest (most common) letter


# A*: guess letter with highest entropy (information gain),
# which best reduces the uncertainty in the possible word list
def a_star_guess(possible_words, guessed):
    def entropy(letter):
        groups = {}
        # Group words by the pattern of where the letter appears
        for word in possible_words:
            key = "".join(['1' if l == letter else '0' for l in word])
            groups.setdefault(key, []).append(word)
        total = len(possible_words)
        # Calculate entropy 
        return -sum((len(g)/total) * math.log2(len(g)/total) for g in groups.values())

    # Filter out already guessed letters
    candidate_letters = [l for l in string.ascii_lowercase if l not in guessed]
    if not candidate_letters:
        return None

    # Score each letter based on entropy
    scores = {l: entropy(l) for l in candidate_letters}
    max_score = max(scores.values())

    # Get all letters with the highest entropy
    best_letters = [l for l in scores if abs(scores[l] - max_score) < 1e-6]

    # Break ties by choosing the one that appears most in the current possible words
    if len(best_letters) > 1:
        word_blob = "".join(possible_words)
        freq = {l: word_blob.count(l) for l in best_letters}
        return max(freq, key=freq.get)

    return best_letters[0]

Player vs. AI

In [10]:
# Popup for player to choose one of five random words for the AI to guess
def choose_word_popup(difficulty):
    # Select word length range based on difficulty
    if difficulty == "easy":
        filtered = [w for w in word_list if len(w.replace(" ", "").replace("-", "")) <= 10]
    elif difficulty == "medium":
        filtered = [w for w in word_list if 11 <= len(w.replace(" ", "").replace("-", "")) <= 14]
    else:  # hard
        filtered = [w for w in word_list if len(w.replace(" ", "").replace("-", "")) > 14]

    if len(filtered) < 5:
        messagebox.showinfo("Not enough words", f"Not enough words available for difficulty: {difficulty.title()}")
        return None

    options = random.sample(filtered, 5)

    def submit_choice():
        nonlocal selected
        selected = word_var.get()
        chooser.destroy()

    chooser = tk.Tk()
    chooser.title("Choose a Word")
    word_var = tk.StringVar(value=options[0])

    tk.Label(chooser, text="Please pick a word for the AI to guess:", font=("Helvetica", 12)).pack(pady=5)
    for w in options:
        tk.Radiobutton(chooser, text=w.upper(), variable=word_var, value=w).pack(anchor="w")

    tk.Button(chooser, text="Submit", command=submit_choice).pack()
    selected = None
    chooser.mainloop()

    return selected


# Player vs AI
def player_vs_ai(difficulty):
    secret_word = choose_word_popup(difficulty)
    if not secret_word:
        return

    guesses = []
    possible_words = [w for w in word_list if len(w) == len(secret_word)]
    mistakes = 0

    window = tk.Tk()
    window.title(f"PvAI Mode - {difficulty.upper()}")
    window.geometry("900x600")

    canvas = tk.Canvas(window, width=200, height=200)
    canvas.pack()

    word_display = tk.StringVar()
    word_bank_display = tk.StringVar()

    tk.Label(window, textvariable=word_display, font=("Helvetica", 24)).pack()
    tk.Label(window, text="Word Bank:", font=("Helvetica", 12)).pack()
    tk.Label(window, textvariable=word_bank_display, font=("Helvetica", 12)).pack()

    def ai_turn():
        nonlocal mistakes, possible_words

        if difficulty == "easy":
            guess = greedy_guess(possible_words, guesses)
        elif difficulty == "medium":
            guess = ucs_guess(possible_words, guesses)
        else:
            guess = a_star_guess(possible_words, guesses)

        if guess and guess not in guesses:
            guesses.append(guess)
            if guess not in secret_word:
                mistakes += 1

            update_display(word_display, secret_word, guesses)
            word_bank_display.set(", ".join(guesses))
            draw_hangman(canvas, mistakes)

            # Narrow down word pool based on guesses
            possible_words = [
                w for w in possible_words if all(
                    (c == w[i] if c not in "- " and c in guesses else True)
                    for i, c in enumerate(secret_word)
                )
            ]

            if all(c in guesses or c in "- " for c in secret_word if c.isalpha()):
                messagebox.showinfo("AI Wins", f"The AI guessed the word: {secret_word.upper()}")
                window.destroy()
                launch_game()
            elif mistakes >= max_attempts:
                messagebox.showinfo("AI Loses", f"The AI failed. Word was: {secret_word.upper()}")
                window.destroy()
                launch_game()

    tk.Button(window, text="Next AI Guess", command=ai_turn, font=("Helvetica", 14)).pack(pady=10)
    update_display(word_display, secret_word, guesses)
    window.mainloop()

    def ai_turn():
        nonlocal mistakes, possible_words

        if difficulty == "easy":
            guess = greedy_guess(possible_words, guesses)
        elif difficulty == "medium":
            guess = ucs_guess(possible_words, guesses)
        else:
            guess = a_star_guess(possible_words, guesses)

        if guess and guess not in guesses:
            guesses.append(guess)
            if guess not in secret_word:
                mistakes += 1

            update_display(word_display, secret_word, guesses)
            word_bank_display.set(", ".join(guesses))
            draw_hangman(canvas, mistakes)

            # Narrow down word pool based on guesses
            possible_words = [
                w for w in possible_words if all(
                    (c == w[i] if c not in "- " and c in guesses else True)
                    for i, c in enumerate(secret_word)
                )
            ]

            if all(c in guesses or c in "- " for c in secret_word if c.isalpha()):
                messagebox.showinfo("AI Wins", f"The AI guessed the word: {secret_word.upper()}")
                window.destroy()
                launch_game()
            elif mistakes >= max_attempts:
                messagebox.showinfo("AI Loses", f"The AI failed. Word was: {secret_word.upper()}")
                window.destroy()
                launch_game()

    tk.Button(window, text="Next AI Guess", command=ai_turn, font=("Helvetica", 14)).pack(pady=10)
    update_display(word_display, secret_word, guesses)
    window.mainloop()


Player vs Player

In [12]:
# Main game mode: Player vs Player
def player_vs_player():
    secret_word = simpledialog.askstring("Player 1", "Enter a secret word:")
    if not secret_word or not secret_word.isalpha():
        return

    secret_word = secret_word.lower()
    guesses = []
    mistakes = 0

    window = tk.Tk()
    window.title("Player vs Player")
    window.geometry("900x600")

    canvas = tk.Canvas(window, width=200, height=200)
    canvas.pack()

    word_display = tk.StringVar()
    word_bank_display = tk.StringVar()

    tk.Label(window, textvariable=word_display, font=("Helvetica", 24)).pack()
    tk.Label(window, text="Word Bank:", font=("Helvetica", 12)).pack()
    tk.Label(window, textvariable=word_bank_display, font=("Helvetica", 12)).pack()

    entry = tk.Entry(window, font=("Helvetica", 14))
    entry.pack()

    # Player 2 guesses
    def submit_guess():
        nonlocal mistakes
        guess = entry.get().lower()
        entry.delete(0, tk.END)

        if guess in guesses:
            messagebox.showinfo("Try again", "Letter already guessed, try again")
        elif guess and guess in string.ascii_lowercase:
            guesses.append(guess)
            if guess not in secret_word:
                mistakes += 1

            update_display(word_display, secret_word, guesses)
            word_bank_display.set(", ".join(guesses))
            draw_hangman(canvas, mistakes)

            if all(c in guesses for c in secret_word):
                messagebox.showinfo("Player 2 Wins", "Guessed the word!")
                window.destroy()
                launch_game()
            elif mistakes >= max_attempts:
                messagebox.showinfo("Player 2 Loses", f"Too many mistakes. Word was: {secret_word.upper()}")
                window.destroy()
                launch_game()

    tk.Button(window, text="Submit Guess", command=submit_guess, font=("Helvetica", 14)).pack(pady=10)
    update_display(word_display, secret_word, guesses)
    window.mainloop()


Main menu

In [14]:
# Launch the main menu screen
def launch_game():
    root = tk.Tk()
    root.title("Sustainability Hangman")
    root.geometry("900x600")

    mode_var = tk.StringVar(value="pvai")
    difficulty_var = tk.StringVar(value="Easy: Greedy highest frequency unguessed letter + shorter words")

    tk.Label(root, text="Choose Game Mode", font=("Helvetica", 14)).pack(pady=10)
    tk.Radiobutton(root, text="Player vs AI", variable=mode_var, value="pvai").pack()
    tk.Radiobutton(root, text="Player vs Player", variable=mode_var, value="pvp").pack()

    tk.Label(root, text="AI Difficulty (only for PvAI):", font=("Helvetica", 12)).pack(pady=10)
    ttk.Combobox(
        root,
        textvariable=difficulty_var,
        values=[
            "Easy: Greedy highest frequency unguessed letter + shorter words",
            "Medium UCS cost is inverse of letter frequency + medium words",
            "Hard A*: cost + heuristic + longer words"
        ],
        width=55
    ).pack()

    def start_mode():
        root.destroy()
        if mode_var.get() == "pvai":
            if "Easy" in difficulty_var.get():
                player_vs_ai("easy")
            elif "Medium" in difficulty_var.get():
                player_vs_ai("medium")
            else:
                player_vs_ai("hard")
        else:
            player_vs_player()

    tk.Button(
        root,
        text="Start Game",
        command=start_mode,
        font=("Helvetica", 14),
        bg="green",
        fg="white"
    ).pack(pady=30)

    root.mainloop()

# Start the game
launch_game()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\brezy\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\brezy\AppData\Local\Temp\ipykernel_9028\215723096.py", line 34, in start_mode
    player_vs_ai("hard")
  File "C:\Users\brezy\AppData\Local\Temp\ipykernel_9028\4258816435.py", line 137, in player_vs_ai
    tk.Button(window, text="Next AI Guess", command=ai_turn, font=("Helvetica", 14)).pack(pady=10)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\brezy\anaconda3\Lib\tkinter\__init__.py", line 2737, in __init__
    Widget.__init__(self, master, 'button', cnf, kw)
  File "C:\Users\brezy\anaconda3\Lib\tkinter\__init__.py", line 2659, in __init__
    self.tk.call(
_tkinter.TclError: can't invoke "button" command: application has been destroyed
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Us